## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


## загрузка данных

In [1]:
!pip install pyconll

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [75]:
import pyconll

import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [32]:
MY_DRIVE_PATH = '/content/drive/My\ Drive/Colab\ Notebooks/data/nlp-datasets'

TRAIN_PATH = MY_DRIVE_PATH + '/ru_syntagrus-ud-train.conllu'
DEV_PATH = MY_DRIVE_PATH + '/ru_syntagrus-ud-dev.conllu'

In [33]:
!wget -O $TRAIN_PATH https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O $DEV_PATH https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-08-16 11:56:48--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘/content/drive/My Drive/Colab Notebooks/data/nlp-datasets/ru_syntagrus-ud-train.conllu’

/content/drive/My D 100%[===================>]  77.28M  50.7MB/s    in 1.5s    

2021-08-16 11:56:50 (50.7 MB/s) - ‘/content/drive/My Drive/Colab Notebooks/data/nlp-datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-08-16 11:56:50--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 18

In [36]:
full_train = pyconll.load_from_file(TRAIN_PATH.replace('\\', ''))
full_test = pyconll.load_from_file(DEV_PATH.replace('\\', ''))

In [37]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [84]:
fdata_train = [[(token.form, token.upos) for token in sent] for sent in full_test[:]]

fdata_test = [[(token.form, token.upos) for token in sent] for sent in full_test[:]]

fdata_sent_test = [[token.form for token in sent] for sent in full_test[:]]
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)

print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [70]:
train_texts = [' '.join(token.form for token in sent) for sent in full_train]
test_texts = [' '.join(token.form for token in sent) for sent in full_test]

train_labels = [' '.join(token.form for token in sent) for sent in full_train]
test_labels = [' '.join(token.form for token in sent) for sent in full_test]

print('\n'.join(train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

### Default Tagger

In [88]:
default_tagger = nltk.DefaultTagger('NOUN')

default_tagger.evaluate(fdata_test)

0.23568564014423887

### Unigram Tagger

In [89]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.973317494018131

### Bigram Tagger

In [90]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

0.9815404576551073

### Trigram Tagger

In [92]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

0.9862501263775149

In [93]:
trigram_tagger.tag(fdata_sent_test[2])

[('Это', 'PRON'),
 ('связано', 'VERB'),
 ('с', 'ADP'),
 ('тем', 'PRON'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('работа', 'NOUN'),
 ('каких-то', 'DET'),
 ('инструкций', 'NOUN'),
 ('алгоритма', 'NOUN'),
 ('может', 'VERB'),
 ('быть', 'AUX'),
 ('зависима', 'ADJ'),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('инструкций', 'NOUN'),
 ('или', 'CCONJ'),
 ('результатов', 'NOUN'),
 ('их', 'DET'),
 ('работы', 'NOUN'),
 ('.', 'PUNCT')]

### Комбинация тэггеров

In [94]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tags = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tags.evaluate(fdata_test)

0.9844808411687392

### Векторайзеры 

In [95]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [96]:
le = LabelEncoder()

train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

print(f'Classes: {le.classes_} \nNumber of classes: {len(le.classes_)}')

Classes: ['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NO_TAG' 'NUM' 'PART'
 'PRON' 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X'] 
Number of classes: 18


In [97]:
for vectorizer in [CountVectorizer, 
                   HashingVectorizer, 
                   TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.9693071142115728
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.9701159303070132
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9701243554746739


In [99]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [100]:
params = {
    "booster": "gbtree",
    "objective": "multi:softmax",
    "num_class": 18,
    "eval_metric": "auc",
    "learning_rate": 0.1,
    "reg_lambda": 100,
    "max_depth": 4,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    coder = vectorizer(ngram_range=(1, 5), analyzer='word', stop_words=stop_words)
    
    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    dtrain = xgb.DMatrix(data=X_train, label=train_enc_labels)
    dtest = xgb.DMatrix(data=X_test, label=test_enc_labels)

    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=10,
        maximize=True)
    
    pred = model.predict(dtest)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.24604017119940688
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.24604017119940688
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.24604017119940688



Из одиночных теггеров лучший результат показали биграмный (0.982) и триграммный (0.986) теггеры.

Комбинация теггеров [UnigramTagger, BigramTagger, TrigramTagger] поесередине между биграмным и триграммным тегенрами

Векторайзеры работают чуть хуже теггеров. 
При этом все три векторайзера почти не отличаются по точности в сочетании с логистической регрессией.

Бустинги работают занчительно хуже. 

# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [101]:
!pip install corus

     |████████████████████████████████| 83 kB 1.4 MB/s 


In [127]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 1.3 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=5950d066e01e71b8d1d206ad149379c7892f9b192bcc95385d474650056a0a7e
  Stored in directory: /tmp/pip-ephem-wheel-cache-5e8x246v/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [128]:
import corus
from corus import load_ne5

import re
import nltk

nltk.download('ieer')

import spacy
from spacy import displacy
import en_core_web_md

[nltk_data] Downloading package ieer to /root/nltk_data...
[nltk_data]   Package ieer is already up-to-date!


In [103]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-08-16 14:49:10--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   442KB/s    in 4.2s    

2021-08-16 14:49:14 (442 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [104]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [105]:
!rm collection5.zip

In [107]:
!ls

Collection5  drive  sample_data


In [108]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='1170',
    text='\r\nСобянин собрал необходимые подписи избирателей в свою поддержку\r\n\r\n\r\nОНФ не будет требовать ухода Исаева при пересмотре решения об ИП\r\n\r\nРанее участники ОНФ поставить вопрос о соответствии занимаемой должности главы комитета ГД по труду и социальной политике Андрея Исаева из-за законопроекта о страховых взносах для ИП в размере 2 МРОТ.\r\n\r\n Участники Народного фронта не будут настаивать на снятии Андрея Исаева с поста председателя комитета Госдумы по труду, социальной политике и делам ветеранов, если комитет пересмотрит решение по законопроекту о снижении страховых взносов для индивидуальных предпринимателей (ИП), заявил журналистам сопредседатель центрального штаба ОНФ Александр Галушка.\r\n\r\nРанее участники ОНФ — "Деловая Россия" и "Опора России" — просили депутатов поставить вопрос о соответствии занимаемой должности главы комитета нижней палаты парламента по труду и социальной политике Андрея Исаева из-за законопроекта о страхо

процедуры обработки взять из вебинарного ноутбука

In [116]:
document = next(records).text
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

nltk.pos_tag(nltk.word_tokenize(document))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


[('Прогноз', 'JJ'),
 ('2012', 'CD'),
 (':', ':'),
 ('Кто', 'NN'),
 ('пойдет', 'NNP'),
 ('в', 'NNP'),
 ('президенты', 'NNP'),
 ('России', 'NNP'),
 ('За', 'NNP'),
 ('год', 'NNP'),
 ('до', 'NNP'),
 ('президентских', 'NNP'),
 ('выборов', 'NNP'),
 ('известные', 'NNP'),
 ('политологи', 'NNP'),
 ('специально', 'NNP'),
 ('для', 'NNP'),
 ('РБК', 'NNP'),
 ('сделали', 'NNP'),
 ('прогнозы', 'NNP'),
 ('относительно', 'NNP'),
 ('того', 'NNP'),
 (',', ','),
 ('кто', 'NNP'),
 ('будет', 'NNP'),
 ('претендовать', 'NNP'),
 ('на', 'NNP'),
 ('президентский', 'NNP'),
 ('пост', 'NNP'),
 ('в', 'NNP'),
 ('марте', 'VBD'),
 ('2012г', 'CD'),
 ('.', '.'),
 ('Главная', 'VB'),
 ('интрига', 'JJ'),
 ('–', 'NNP'),
 ('кандидат', 'NNP'),
 ('от', 'NNP'),
 ('партии', 'NNP'),
 ('власти', 'NNP'),
 ('.', '.'),
 ('Стартуют', 'VB'),
 ('``', '``'),
 ('темные', 'JJ'),
 ('лошадки', 'NN'),
 ("''", "''"),
 ('После', 'CC'),
 ('отчета', 'NNP'),
 ('В.Путина', 'NNP'),
 ('о', 'NNP'),
 ('деятельности', 'NNP'),
 ('правительства', 'NNP'),
 

In [121]:
[(' '.join(c[0] for c in chunk), chunk.label()) 
            for chunk in nltk.ne_chunk(
                nltk.pos_tag(nltk.word_tokenize(document))
                ) if hasattr(chunk, 'label')
]

[('Кто', 'PERSON'),
 ('России За', 'PERSON'),
 ('Владимира Владимировича Путина', 'ORGANIZATION'),
 ('Полишинеля', 'ORGANIZATION'),
 ('Центра', 'PERSON'),
 ('Алексей Мухин', 'PERSON'),
 ('Путина', 'PERSON'),
 ('Госдуму', 'PERSON'),
 ('Владимир Путин', 'PERSON'),
 ('Сергей Собянин', 'PERSON'),
 ('Сергей Нарышкин', 'PERSON'),
 ('Шувалов', 'PERSON'),
 ('Сергея Михайловича Миронова', 'PERSON'),
 ('Андрей Богданов', 'PERSON'),
 ('Алексею Навальному', 'PERSON'),
 ('Центра', 'PERSON'),
 ('Шатилов', 'PERSON'),
 ('Дмитрий Медведев', 'PERSON'),
 ('Урнов', 'PERSON'),
 ('Ситуация', 'PERSON'),
 ('Скорей', 'PERSON'),
 ('Будет', 'PERSON'),
 ('Дмитрий Медведев', 'PERSON'),
 ('Кремлем', 'PERSON'),
 ('Алексея Навального', 'PERSON'),
 ('Виноградов', 'PERSON'),
 ('Центра', 'PERSON'),
 ('Алексей Макаркин', 'PERSON'),
 ('Центра', 'PERSON'),
 ('Сейчас', 'PERSON')]

### Spacy

In [129]:
nlp = en_core_web_md.load()
doc = nlp(document)

displacy.render(doc, style="dep") # (1)
displacy.render(doc, style="ent") # (2) 

'<div class="entities" style="line-height: 2.5; direction: ltr">\n<mark class="entity" style="background: #aa9cfc; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Прогноз\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">PERSON</span>\n</mark>\n 2012: \n<mark class="entity" style="background: #aa9cfc; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Кто пойдет\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">PERSON</span>\n</mark>\n в президенты России\r</br>\r</br>За год до президентских выборов известные политологи специально для РБК сделали прогнозы относительно того, \n<mark class="entity" style="background: #aa9cfc; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; bord

In [132]:
!pip install IPython

In [141]:
import IPython
IPython.display.HTML(filename='/content/q.html')

In [143]:
!pip install razdel

In [144]:
from razdel import tokenize

words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):       
        result = 'None'        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
        words.append([token.text, result])        
    words_docs.extend(words)

In [145]:
import pandas as pd

df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words['tag'].value_counts()

None        216535
PER          20935
ORG          13566
LOC           4511
GEOPOLIT      4338
MEDIA         2468
Name: tag, dtype: int64

In [146]:
df_words.head()

,word,tag
0,РФ,GEOPOLIT
1,получит,None
2,от,None
3,Фонда,ORG
4,МСФО,ORG


In [148]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, Conv2D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

In [149]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [150]:
train_x.apply(len).max(axis=0)

55

In [151]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [152]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [153]:
def custom_standardization(input_data):
    return input_data

def data_preparation(train_data, seq_len=1, vocab_size = 30000):    

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [154]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [155]:
vocab_size = 30000
vectorize_layer = data_preparation(train_data, seq_len = 1, vocab_size=vocab_size)

mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12298/12298 [==============================] - 310s 25ms/step - loss: 0.2847 - accuracy: 0.9176 - val_loss: 0.2151 - val_accuracy: 0.9357
Epoch 2/5
12298/12298 [==============================] - 310s 25ms/step - loss: 0.1219 - accuracy: 0.9636 - val_loss: 0.2463 - val_accuracy: 0.8919
Epoch 3/5
12298/12298 [==============================] - 314s 26ms/step - loss: 0.1078 - accuracy: 0.9661 - val_loss: 0.2216 - val_accuracy: 0.9388
Epoch 4/5
12298/12298 [==============================] - 306s 25ms/step - loss: 0.1030 - accuracy: 0.9670 - val_loss: 0.2501 - val_accuracy: 0.9270
Epoch 5/5
12298/12298 [==============================] - 309s 25ms/step - loss: 0.1001 - accuracy: 0.9676 - val_loss: 0.2749 - val_accuracy: 0.9251


In [156]:
vocab_size = 30000
vectorize_layer = data_preparation(train_data, seq_len = 5, vocab_size=vocab_size)

mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12298/12298 [==============================] - 305s 25ms/step - loss: 0.2926 - accuracy: 0.9147 - val_loss: 0.2196 - val_accuracy: 0.9373
Epoch 2/5
12298/12298 [==============================] - 305s 25ms/step - loss: 0.1256 - accuracy: 0.9630 - val_loss: 0.3786 - val_accuracy: 0.8919
Epoch 3/5
12298/12298 [==============================] - 305s 25ms/step - loss: 0.1095 - accuracy: 0.9659 - val_loss: 0.2342 - val_accuracy: 0.8922
Epoch 4/5
12298/12298 [==============================] - 305s 25ms/step - loss: 0.1038 - accuracy: 0.9668 - val_loss: 0.2398 - val_accuracy: 0.8919
Epoch 5/5
12298/12298 [==============================] - 304s 25ms/step - loss: 0.1009 - accuracy: 0.9673 - val_loss: 0.2944 - val_accuracy: 0.8892


In [157]:
vectorize_layer = data_preparation(train_data, seq_len = 10, vocab_size=vocab_size)

mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12298/12298 [==============================] - 311s 25ms/step - loss: 0.2946 - accuracy: 0.9141 - val_loss: 0.2160 - val_accuracy: 0.9376
Epoch 2/5
12298/12298 [==============================] - 309s 25ms/step - loss: 0.1244 - accuracy: 0.9631 - val_loss: 0.2181 - val_accuracy: 0.9392
Epoch 3/5
12298/12298 [==============================] - 306s 25ms/step - loss: 0.1088 - accuracy: 0.9659 - val_loss: 0.2148 - val_accuracy: 0.9391
Epoch 4/5
12298/12298 [==============================] - 313s 25ms/step - loss: 0.1035 - accuracy: 0.9669 - val_loss: 0.2469 - val_accuracy: 0.8898
Epoch 5/5
12298/12298 [==============================] - 296s 24ms/step - loss: 0.1005 - accuracy: 0.9674 - val_loss: 0.2729 - val_accuracy: 0.8883


Длина входной последовательности слабо влияет на результат.